In [77]:
import copy
def returnBoard(filename, rows, columns):
    # SRC: http://stackoverflow.com/questions/3277503/how-to-read-a-file-line-by-line-into-a-list
    with open(filename) as f:
        content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        content = [x.split() for x in content] 
    board = []
    for row in range(rows):
        currentRow = []
        for column in range(columns):
            if((content[row][column])=='-'):
                value = 0
            else:
                value = int(content[row][column])
            currentRow.append(value)
        board.append(currentRow)
        # print currentRow
    return board
    
    
class SudokuPuzzle:
    def __init__(self, possibleValues, filename, board=None, initialize=True, rows=9, columns=9, boxRows=3, boxColumns=3):
        if(initialize):
            self.variableSize = rows * columns
            self.possibleValues = possibleValues
            self.rows = rows
            self.columns = columns
            self.boxRows = boxRows
            self.boxColumns = boxColumns
            if(board==None):
                self.board = returnBoard(filename,rows,columns)
            else:
                self.board = board
            # print "Creating domainVariables"
            self.updateDomainVariables()
            # print self.domainVariables
            
    def rowAllDiff(self, row):
        dictV = {}
        for column in range(self.columns):
            if(self.board[row][column] in dictV):
                # print "Row ALL DIFF Failed"
                return False
            else:
                if(self.board[row][column]!=0):
                    dictV[self.board[row][column]] = 1
        return True
    
    def columnAllDiff(self, column):
        dictV = {}
        for row in range(self.rows):
            if(self.board[row][column] in dictV):
                # print "Col ALL DIFF Failed"
                return False
            else:
                if(self.board[row][column]!=0):
                    dictV[self.board[row][column]] = 1
        return True
    
    def updateDomainAfterAssignment(self, rcIndex, value):
        neighbours = self.findNeighbours(rcIndex)
        for neighbour in neighbours:
            nRow, nCol = neighbour
            if(self.board[nRow][nCol]==0):
                if(value in self.variableValuesAvailable[nRow][nCol]):
                    self.variableValuesAvailable[nRow][nCol].remove(value)
        
    
    def boxRemoveColumnAndRowValues(self):
        boxrowstart=[0,3,6]
        boxcolstart=[0,3,6]
        
        for arrayrowidx in range(3):
            for arraycolidx in range(3):
                BoxMap = {}
                IndicesOfBox = []
                for i in range(boxrowstart[arrayrowidx],boxrowstart[arrayrowidx]+3):
                    for j in range(boxcolstart[arraycolidx],boxcolstart[arraycolidx]+3):
                        IndicesOfBox.append((i,j))
                        if self.board[i][j]==0:
                            setValues = copy.deepcopy(self.variableValuesAvailable[i][j])
                            setValues = frozenset(setValues)
                            if(setValues not in BoxMap):
                                
                                BoxMap[setValues] = [(i,j)]
                            else:
                                BoxMap[setValues].append((i,j))
                print BoxMap
                for key in BoxMap:
                    print "CHECKING KEY - CAME IN"
                    print key
                    print BoxMap[key]
                    if(len(BoxMap[key]) == len(key) and len(key)==2):
                        # JUST DOING FOR 2 Values Similar
                        r1,c1 = BoxMap[key][0]
                        r2,c2 = BoxMap[key][1]
                        print r1,c1,r2,c2
                        rowOnly = False
                        colOnly = False
                        if(r1 == r2):
                            rowOnly = True
                        if(c1 == c2):
                            colOnly = True
                        print "RowOnly"+str(rowOnly)
                        print "ColOnly"+str(colOnly)
                        if(not(rowOnly or colOnly)):
                            print "Failure"
                        for rcIndex in BoxMap[key]:
                            neighbours = self.findNeighbours(rcIndex)
                            for neighbour in neighbours:
                                nRow, nCol = neighbour
                                if(self.board[nRow][nCol]==0 and (neighbour not in IndicesOfBox)):
                                    if(rowOnly and nRow == r1):
                                        print "Checking Neighbour %d, %d" %(nRow,nCol)
                                        for value in key:
                                            if(value in self.variableValuesAvailable[nRow][nCol]):
                                                print self.variableValuesAvailable[nRow][nCol]
                                                print "Removing Value From Neighbour %d, %d" %(nRow,nCol)
                                                self.variableValuesAvailable[nRow][nCol].remove(value)
                                                print self.variableValuesAvailable[nRow][nCol]
                                                # remove from other columns/rows not in this box
                                    elif(colOnly and nCol == c1):
                                        print "Checking Neighbour %d, %d" %(nRow,nCol)
                                        for value in key:
                                            if(value in self.variableValuesAvailable[nRow][nCol]):
                                                print self.variableValuesAvailable[nRow][nCol]
                                                print "Removing Value From Neighbour %d, %d" %(nRow,nCol)
                                                self.variableValuesAvailable[nRow][nCol].remove(value)
                                                print self.variableValuesAvailable[nRow][nCol]
                                                # remove from other columns/rows not in this box
    
    def boxAllDiff(self, r, c):
        
        dictV = {}
        boxRowStart = self.boxRows*(r/self.boxRows)
        boxColumnStart = self.boxColumns*(c/self.boxColumns)
        boxRowEnd = boxRowStart + self.boxRows
        boxColumnEnd = boxColumnStart + self.boxColumns
        # print "CHECKING BOX (%d, %d) " %(r,c) 
        for row in range(boxRowStart, boxRowEnd):
            for column in range(boxColumnStart, boxColumnEnd):
                # print (row,column)
                if(self.board[row][column] in dictV):
                    # self.printBoard()
                    # print "Board ALL DIFF Failed"
                    return False
                else:
                    if(self.board[row][column]!=0):
                        dictV[self.board[row][column]] = 1
        return True
    
    def checkConstraints(self, row, column):
        toCheckVal = self.board[row][column]
        return self.rowAllDiff(row) and self.columnAllDiff(column) and self.boxAllDiff(row, column)
    
    def nakedpairs(self):
        for row in range(self.rows):
            for k in range(1,10):
                found=False
                for column in range(self.columns):
                    if(self.board[row][column]==k):
                        found=True
                        break
                if found==False:
                    count=0
                    for column in range(self.columns):
                        if(self.board[row][column] == 0 and k in self.variableValuesAvailable[row][column]):
                            pos=column
                            count+=1
                    if count==1:
                        self.assignment(row,pos,k)
        for column in range(self.columns):
            for k in range(1,10):
                found=False
                for row in range(self.rows):
                    if(self.board[row][column]==k):
                        found=True
                        break
                if found==False:
                    count=0
                    for row in range(self.rows):
                        if(self.board[row][column] == 0 and k in self.variableValuesAvailable[row][column]):
                            pos=row
                            count+=1
                    if count==1:
                        self.assignment(pos,column,k)
    
        boxrowstart=[0,3,6]
        boxcolstart=[0,3,6]
        for arrayrowidx in range(3):
            for arraycolidx in range(3):
                
                for k in range(1,10):
                    found=False
                    for i in range(boxrowstart[arrayrowidx],boxrowstart[arrayrowidx]+3):
                        for j in range(boxcolstart[arraycolidx],boxcolstart[arraycolidx]+3):
                            if self.board[i][j]==k:
                                found=True
                    if found==False:
                        count=0
                        for i in range(boxrowstart[arrayrowidx],boxrowstart[arrayrowidx]+3):
                            for j in range(boxcolstart[arraycolidx],boxcolstart[arraycolidx]+3):
                                if self.board[i][j] == 0 and k in self.variableValuesAvailable[i][j]:
                                    posx,posy=i,j
                                    count+=1
                        if count==1:
                            self.assignment(posx,posy,k)
                        
                        
   
        
    def reachedGoal(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.board[row][column]==0):
                    return False
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.checkConstraints(row,column)):
                    continue
                else:
                    return None
        return True
    
    def assignment(self, row, column, value):
        self.board[row][column] = value
        self.updateDomainVariables()
    
    def backtrackAssignment(self, row, column, value):
        self.board[row][column] = value
        
    def updateDomainVariables(self):
        domain = {}
        for value in self.possibleValues:
            domain[value] = 1;
        boxAvailableValues = []
        rowAvailableValues = []
        columnAvailableValues = []
        # copy.copy(domain)
        for row in range(self.rows):
            rowAvailableValues.append(copy.deepcopy(domain))
        for column in range(self.columns):
            columnAvailableValues.append(copy.deepcopy(domain))
        
        totalBoxes = (self.rows * self.columns)/(self.boxRows*self.boxColumns)
        for box in range(totalBoxes):
            boxAvailableValues.append(copy.deepcopy(domain))
            
        variableDomain = []
        # Add used values 
        for row in range(self.rows):
            variableDomain.append([])
            for column in range(self.columns):
                if(self.board[row][column]!=0):
                    boxAvailableValues[((3*(row/3))+(column/3))][self.board[row][column]] = 0
                    rowAvailableValues[row][self.board[row][column]] = 0
                    columnAvailableValues[column][self.board[row][column]] = 0

        for row in range(self.columns):
            for column in range(self.columns):  
                if(self.board[row][column]==0):
                    variableDomain[row].append(self.findPositionsPossibleValues(row, column, boxAvailableValues, rowAvailableValues, columnAvailableValues))
                else:
                    variableDomain[row].append(self.board[row][column])
        # print variableDomain, boxAvailableValues, rowAvailableValues, columnAvailableValues
        self.variableValuesAvailable = variableDomain
        self.boxValuesAvailable = boxAvailableValues
        self.rowValuesAvailable = rowAvailableValues
        self.columnValuesAvailable = columnAvailableValues
        return variableDomain, boxAvailableValues, rowAvailableValues, columnAvailableValues
        
    def findPositionsPossibleValues(self, row, column, boxAvailableValues, rowAvailableValues, columnAvailableValues):
        BoxIndex = ((3*(row/3))+(column/3))
        boxHash = boxAvailableValues[BoxIndex]
        boxValueSet = set({k: v for k, v in boxHash.iteritems() if boxHash[k]==1})
        rowHash = rowAvailableValues[row]
        rowValueSet = set({k: v for k, v in rowHash.iteritems() if rowHash[k]==1})
        columnHash = columnAvailableValues[column]
        columnValueSet = set({k: v for k, v in columnHash.iteritems() if columnHash[k]==1})
        return boxValueSet & columnValueSet & rowValueSet

        
    def printBoard(self):
        for row in range(self.rows):
            print self.board[row]
            
    def printAvailableValues(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.board[row][column]==0):
                    print "Values available for row %d column %d are " %(row, column) 
                    print self.variableValuesAvailable[row][column]
    
    def unAssignedValuesCnt(self):
        unassignedCnt = 0;
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.board[row][column]==0):
                    unassignedCnt+=1;
        return unassignedCnt
    
    def unAssignedValuesBranchFactor(self):
        vCnt = 0.0;
        domLength = 0.0;
        for row in range(self.rows):
            for column in range(self.columns):
                if(self.board[row][column]==0):
                    domLength += len(self.variableValuesAvailable[row][column])
                    vCnt += 1
        return domLength/vCnt;

    
    def reachedFailure(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if( (self.board[row][column]==0) and len(self.variableValuesAvailable[row][column])==0):
                    return True
                if(self.checkConstraints(row,column)):
                    continue
                else:
                    return False
        return False
    
    def updateBoardAndDomainValues(self):
        for row in range(self.rows):
            for column in range(self.columns):
                if((self.board[row][column]==0) and len(self.variableValuesAvailable[row][column])==1):
                    #print "Adding the only elt to the board for %d, %d" %(row,column)
                    self.board[row][column] = list(self.variableValuesAvailable[row][column])[0]
        self.updateDomainVariables()
        #print "Updated Domain Variables"
        #print self.variableValuesAvailable
        needToCallAgain = False
        for row in range(self.rows):
            for column in range(self.columns):
                if((self.board[row][column]==0) and len(self.variableValuesAvailable[row][column])==1):
                    needToCallAgain = True
                    break
        if(needToCallAgain):
            self.updateBoardAndDomainValues()
        return
        
                    
    def revise(self, Xi, Xj):
        revised = False
        # if(Xi[0] == 1 and Xi[1] == 2):
            # pdb.set_trace()
        row, column = Xi
        otherRow, otherColumn = Xj
        if((self.board[row][column])!=0):
            varValuesAvailbleForXi = [self.board[row][column]]
        else:
            varValuesAvailbleForXi = copy.deepcopy(self.variableValuesAvailable[row][column])

        if((self.board[otherRow][otherColumn])!=0):
            varValuesAvailbleForXj = [self.board[otherRow][otherColumn]]
        else:
            varValuesAvailbleForXj = copy.deepcopy(self.variableValuesAvailable[otherRow][otherColumn]) 
    
        # print "---------------------------"
        # print "Revise Function Xi : (%d, %d), Xj : (%d, %d)" %(Xi[0],Xi[1],Xj[0],Xj[1])
        # print "varValuesAvailbleForXi : "
        # print varValuesAvailbleForXi
        # print "varValuesAvailbleForXj :"
        # print varValuesAvailbleForXj
        for valueAvailableForXi in varValuesAvailbleForXi:
            satisfiedValue = False
            
            for otherValueAvailableForXj in varValuesAvailbleForXj:
                if(otherValueAvailableForXj!=valueAvailableForXi):
                    satisfiedValue = True
            if(satisfiedValue == False):
                self.variableValuesAvailable[row][column].remove(valueAvailableForXi)
                revised = True
        # print "Revised Value " + str(revised)
        # print "---------------------------"
        return revised
        
                    
    def addAllConstraintsAvailable(self):
        Constraints = []
        boxRowStart = 0 
        for boxRow in range(self.boxRows):
            boxColumnStart = 0
            for boxColumn in range(self.boxColumns):
                for value in self.boxValueCombinations(boxRowStart, boxColumnStart):
                    Constraints.append(value)
                boxColumnStart+=self.boxColumns
            boxRowStart += self.boxRows
        
        for row in range(self.rows):
            ColumnVariables = []
            for column in range(self.columns):
                ColumnVariables.append((row,column))
            for value in itertools.combinations(ColumnVariables, 2):
                Constraints.append(value)
        
        for column in range(self.columns):
            RowVariables = []
            for row in range(self.rows):
                RowVariables.append((row,column))
            for value in itertools.combinations(RowVariables, 2):
                Constraints.append(value)
        
        
        return Constraints #itertools.combinations(Variables, 2)
    
    def boxValueCombinations(self, r, c):
        boxVariables = []
        boxRowStart = self.boxRows*(r/self.boxRows)
        boxColumnStart = self.boxColumns*(c/self.boxColumns)
        boxRowEnd = boxRowStart + self.boxRows
        boxColumnEnd = boxColumnStart + self.boxColumns
        for row in range(boxRowStart, boxRowEnd):
            for column in range(boxColumnStart, boxColumnEnd):
                boxVariables.append((row, column))
        return itertools.combinations(boxVariables, 2)

    def findNeighbours(self, rcInd):
        r,c = rcInd
        Neighbours = set()
        for row in range(self.rows):
            Neighbours.add((row, c))
        for column in range(self.columns):
            Neighbours.add((r, column))
        
        boxRowStart = self.boxRows*(r/self.boxRows)
        boxColumnStart = self.boxColumns*(c/self.boxColumns)
        boxRowEnd = boxRowStart + self.boxRows
        boxColumnEnd = boxColumnStart + self.boxColumns
        for row in range(boxRowStart, boxRowEnd):
            for column in range(boxColumnStart, boxColumnEnd):
                Neighbours.add((row, column))

        Neighbours.remove((r,c))
#         print "Neighbours of (%d %d) ARE" %(r,c)
#         print Neighbours
        return list(Neighbours)


In [2]:
count=0
searchcount=0
def mrv(puzzle):
    minval=100
    for row in range(puzzle.rows):
        for column in range(puzzle.columns):
            if((puzzle.board[row][column]==0) and len(puzzle.variableValuesAvailable[row][column])<minval ):
                minval=len(puzzle.variableValuesAvailable[row][column])
                rowvalue=row
                columnvalue=column
    
    return rowvalue,columnvalue
def BACKTRACKINGSEARCH(puzzle): 
    return BACKTRACK(puzzle)

def BACKTRACK(puzzle, simple_backtracking=False):
    global count
    global searchcount
   
    # print "**********"
    # puzzle.printBoard()
    # print "**********"
#     if(!simple_backtracking):
        # DOES FORWARD CHECKING, AN INFERENCE METHOD
    
    puzzle.updateDomainVariables()
    #puzzle.nakedpairs()
    reached = puzzle.reachedGoal()
    if(reached == None):
        return False
    if(reached):
        #print "SOLUTION REACHED"
        #puzzle.printBoard()
        # print puzzle.columnAllDiff(1);
        return puzzle
    if(puzzle.reachedFailure()):
        # print "Failure Reached"
        # puzzle.printBoard()
        
        return False
    
    
    rowvalues,columnvalues=mrv(puzzle)
    for row in [rowvalues]:
        for column in [columnvalues]:
            if(puzzle.board[row][column]==0):
                
                # print "Reached %d %d Pos" %(row, column)
                # orderedBestPossibleValues = OrderedDomainValues(puzzle, row, column)
                # puzzle.printAvailableValues()
                # if(len(puzzle.variableValuesAvailable[row][column])==0):
                #    return False
                ValueFromRowColumnSatisfied = False
                possibleValuesInDomain = puzzle.possibleValues
                if len(puzzle.variableValuesAvailable[row][column])!=0:
                    count+=len(puzzle.variableValuesAvailable[row][column])-1
                for value in puzzle.variableValuesAvailable[row][column]:
                    
                    board = copy.deepcopy(puzzle.board)
                    new_puzzle = SudokuPuzzle(possibleValues, filename, board)
                    new_puzzle.assignment(row, column, value)
                    #print "Assigning position (%d %d) a value of %d" %(row, column, value)
                    if(not new_puzzle.checkConstraints(row,column)):
                        # print "CONSTRAINTS FAILED For value : %d" %(value)
                        # new_puzzle.printBoard()
                        continue # assignment was a bad assignment
                    # print "value : %d" %(value)
                    
                    # new_puzzle.printBoard()
                    result = BACKTRACK(new_puzzle)
                    # ValueFromRowColumnSatisfied = result or ValueFromRowColumnSatisfied
                    if(result == False):
                        continue
                    else:
                        ValueFromRowColumnSatisfied = True
                        return result
                if(not ValueFromRowColumnSatisfied):
                    #print "BackTracking for ROW %d COLUMN %d" %(row, column)
                    return False


In [3]:
def SIMPLEBACKTRACK(puzzle):
    global count
    global searchcount
   
    # print "**********"
    # puzzle.printBoard()
    # print "**********"
#     if(!simple_backtracking):
        # DOES FORWARD CHECKING, AN INFERENCE METHOD
    
    #puzzle.updateDomainVariables()
    #puzzle.printBoard()
    #puzzle.nakedpairs()
    reached = puzzle.reachedGoal()
    if(reached == None):
        return False
    if(reached):
        #print "SOLUTION REACHED"
        #puzzle.printBoard()
        # print puzzle.columnAllDiff(1);
        return puzzle
    if(puzzle.reachedFailure()):
        # print "Failure Reached"
        # puzzle.printBoard()
        
        return False
    
    
    #rowvalues,columnvalues=mrv(puzzle)
    for row in range(puzzle.rows):
        for column in range(puzzle.columns):
            #print"Here"
            if(puzzle.board[row][column]==0):
                
                # print "Reached %d %d Pos" %(row, column)
                # orderedBestPossibleValues = OrderedDomainValues(puzzle, row, column)
                # puzzle.printAvailableValues()
                # if(len(puzzle.variableValuesAvailable[row][column])==0):
                #    return False
                ValueFromRowColumnSatisfied = False
                possibleValuesInDomain = puzzle.possibleValues
#                 print row,column
#                 print puzzle.variableValuesAvailable[row][column]
#                 puzzle.printBoard()
                
                if len(possibleValuesInDomain)!=0:
                    count+=len(possibleValuesInDomain)-1
                for value in possibleValuesInDomain:
                    
                    board = copy.deepcopy(puzzle.board)
                    new_puzzle = SudokuPuzzle(possibleValues, filename, board)
                    new_puzzle.backtrackAssignment(row, column, value)
                    #print "Assigning position (%d %d) a value of %d" %(row, column, value)
                    if(not new_puzzle.checkConstraints(row,column)):
                        # print "CONSTRAINTS FAILED For value : %d" %(value)
                        # new_puzzle.printBoard()
                        continue # assignment was a bad assignment
                    # print "value : %d" %(value)
                    
                    # new_puzzle.printBoard()
                    result = SIMPLEBACKTRACK(new_puzzle)
                    # ValueFromRowColumnSatisfied = result or ValueFromRowColumnSatisfied
                    if(result == False):
                        continue
                    else:
                        ValueFromRowColumnSatisfied = True
                        return result
                if(not ValueFromRowColumnSatisfied):
                    #print "BackTracking for ROW %d COLUMN %d" %(row, column)
                    return False

In [4]:
def WATERFALLSEARCH(puzzle): 
    return WATERFALL(puzzle)

def WATERFALL(puzzle, simple_backtracking=False):
    global count
    global searchcount
   
    # print "**********"
    # puzzle.printBoard()
    # print "**********"
#     if(!simple_backtracking):
        # DOES FORWARD CHECKING, AN INFERENCE METHOD
    puzzle.updateBoardAndDomainValues()
    puzzle.nakedpairs()
    reached = puzzle.reachedGoal()
    if(reached == None):
        return False
    if(reached):
        #print "SOLUTION REACHED"
        #puzzle.printBoard()
        # print puzzle.columnAllDiff(1);
        return puzzle
    if(puzzle.reachedFailure()):
        # print "Failure Reached"
        # puzzle.printBoard()
        
        return False
    
    # print count,searchcount
    rowvalues,columnvalues=mrv(puzzle)
    for row in [rowvalues]:
        for column in [columnvalues]:
            if(puzzle.board[row][column]==0):
                
                # print "Reached %d %d Pos" %(row, column)
                # orderedBestPossibleValues = OrderedDomainValues(puzzle, row, column)
                # puzzle.printAvailableValues()
                # if(len(puzzle.variableValuesAvailable[row][column])==0):
                #    return False
                ValueFromRowColumnSatisfied = False
                
                if len(puzzle.variableValuesAvailable[row][column])!=0:
                    count+=len(puzzle.variableValuesAvailable[row][column])-1
                for value in puzzle.variableValuesAvailable[row][column]:
                    
                    board = copy.deepcopy(puzzle.board)
                    new_puzzle = SudokuPuzzle(possibleValues, filename, board)
                    new_puzzle.assignment(row, column, value)
                    # print "Assigning position (%d %d) a value of %d" %(row, column, value)
                    if(not AC_3(new_puzzle, (row, column))):
                        # print "CONSTRAINTS FAILED For value : %d" %(value)
                        # new_puzzle.printBoard()
                        continue # assignment was a bad assignment
                    # print "value : %d" %(value)
                    
                    # new_puzzle.printBoard()
                    result = WATERFALL(new_puzzle)
                    if(result == False):
                        continue
                    else:
                        ValueFromRowColumnSatisfied = True
                        return result
                if(not ValueFromRowColumnSatisfied):
                    #print "BackTracking for ROW %d COLUMN %d" %(row, column)
                    return False


In [5]:
from collections import deque
import itertools

def AC_3(puzzle, assignedIndex):
    # print "AC3 Algorithm ----"
    row, column = assignedIndex
    result = puzzle.addAllConstraintsAvailable()
#     result = puzzle.findNeighbours(assignedIndex)
#     if(result == None):
#         return True
    Deque = deque()
    for value in result:
        Deque.append(value)
    #print Deque
    while(len(Deque)>0):
        Xi, Xj = Deque[0]
        # print Xi,Xj
        Deque.popleft()
        # print "Puzzle is being Checked Xi: "+ str(Xi) + " Xj : " + str(Xj)
        if(puzzle.revise(Xi,Xj)):
            # print "Puzzle is being Revised Xi: "+ str(Xi) + " Xj : " + str(Xj)
            if(puzzle.board[Xi[0]][Xi[1]] ==0 and len(puzzle.variableValuesAvailable[Xi[0]][Xi[1]])==0):
                # print "AC 3 Inference Failure Failed Because of Conflicts"
                return False
            Xi_neighbours = puzzle.findNeighbours(Xi)
            # print "Neighbours of (%d %d) ARE" %(Xi[0],Xi[1])
            # print Xi_neighbours
            if(len(Xi_neighbours)>0):
                for value in (Xi_neighbours):
                    if(value != Xj):
                        Deque.append((value, Xi))

    return True

count=0
searchcount=0

def SolveUsingAC3(puzzle, simple_backtracking=False):
    global count
    global searchcount
   
    # print "**********"
    # puzzle.printBoard()
    # print "**********"
#     if(!simple_backtracking):
        # DOES FORWARD CHECKING, AN INFERENCE METHOD
    puzzle.updateDomainVariables()
    reached = puzzle.reachedGoal()
    if(reached == None):
        return False
    if(reached):
        #print "SOLUTION REACHED"
        #puzzle.printBoard()
        # print puzzle.columnAllDiff(1);
        return puzzle
    if(puzzle.reachedFailure()):
        # print "Failure Reached"
        # puzzle.printBoard()
        
        return False
    
    # print count,searchcount
    rowvalues,columnvalues=mrv(puzzle)
    for row in [rowvalues]:
        for column in [columnvalues]:
            if(puzzle.board[row][column]==0):
                
                # print "Reached %d %d Pos" %(row, column)
                # orderedBestPossibleValues = OrderedDomainValues(puzzle, row, column)
                # puzzle.printAvailableValues()
                # if(len(puzzle.variableValuesAvailable[row][column])==0):
                #    return False
                ValueFromRowColumnSatisfied = False
                
                if len(puzzle.variableValuesAvailable[row][column])!=0:
                    count+=len(puzzle.variableValuesAvailable[row][column])-1
                for value in puzzle.variableValuesAvailable[row][column]:
                    
                    board = copy.deepcopy(puzzle.board)
                    new_puzzle = SudokuPuzzle(possibleValues, filename, board)
                    new_puzzle.assignment(row, column, value)
                    # print "Assigning position (%d %d) a value of %d" %(row, column, value)
                    if(not AC_3(new_puzzle, (row, column))):
                        # print "CONSTRAINTS FAILED For value : %d" %(value)
                        # new_puzzle.printBoard()
                        continue # assignment was a bad assignment
                    # print "value : %d" %(value)
                    
                    # new_puzzle.printBoard()
                    result = SolveUsingAC3(new_puzzle)
                    if(result == False):
                        continue
                    else:
                        ValueFromRowColumnSatisfied = True
                        return result
                if(not ValueFromRowColumnSatisfied):
                    #print "BackTracking for ROW %d COLUMN %d" %(row, column)
                    return False


In [6]:
possibleValues = [1,2,3,4,5,6,7,8,9]

def solveSudokuUsingSimplebackTrack(filename):
    possibleValues = [1,2,3,4,5,6,7,8,9]
    global count
    global searchcount
    count=0
    searchcount=0
    puzzle = SudokuPuzzle(possibleValues, filename)
   
    
    return SIMPLEBACKTRACK(puzzle)


def solveSudokuUsingMRV(filename):
    possibleValues = [1,2,3,4,5,6,7,8,9]
    global count
    global searchcount
    count=0
    searchcount=0
    puzzle = SudokuPuzzle(possibleValues, filename)
    
    return BACKTRACKINGSEARCH(puzzle)


def solveSudokuUsingAC3(filename):
    possibleValues = [1,2,3,4,5,6,7,8,9]
    global count
    global searchcount
    count=0
    searchcount=0
    puzzle = SudokuPuzzle(possibleValues, filename)
    #puzzle.printBoard()
    
    return SolveUsingAC3(puzzle)


def solveSudokuUsingWATERFALL(filename):
    possibleValues = [1,2,3,4,5,6,7,8,9]
    global count
    global searchcount
    count=0
    searchcount=0
    puzzle = SudokuPuzzle(possibleValues, filename)
    
    return WATERFALLSEARCH(puzzle)
def countlister(filename):
    puzzle = solveSudokuUsingSimplebackTrack(filename)
    print "GUESSES FOR SIMPLE BACKTRACK = ",count
    puzzle = solveSudokuUsingMRV(filename)
    print "GUESSES FOR MRV = ",count
    puzzle = solveSudokuUsingAC3(filename)
    print "GUESSES FOR AC3 = ",count
    puzzle = solveSudokuUsingWATERFALL(filename)
    print "GUESSES FOR WATERFALL = ",count
filename=""
numbers=["puzzle/puz-001.txt","puzzle/puz-002.txt","puzzle/puz-010.txt","puzzle/puz-015.txt","puzzle/puz-025.txt","puzzle/puz-026.txt","puzzle/puz-048.txt","puzzle/puz-051.txt","puzzle/puz-062.txt","puzzle/puz-076.txt","puzzle/puz-081.txt","puzzle/puz-082.txt","puzzle/puz-090.txt","puzzle/puz-095.txt","puzzle/puz-099.txt","puzzle/puz-100.txt"]
for i in numbers:
    print i
    global filename
    filename=i
    countlister(i)

puzzle/puz-001.txt
GUESSES FOR SIMPLE BACKTRACK =  424
GUESSES FOR MRV =  0
GUESSES FOR AC3 =  0
GUESSES FOR WATERFALL =  0
puzzle/puz-002.txt
GUESSES FOR SIMPLE BACKTRACK =  608
GUESSES FOR MRV =  0
GUESSES FOR AC3 =  0
GUESSES FOR WATERFALL =  0
puzzle/puz-010.txt
GUESSES FOR SIMPLE BACKTRACK =  704
GUESSES FOR MRV =  0
GUESSES FOR AC3 =  0
GUESSES FOR WATERFALL =  0
puzzle/puz-015.txt
GUESSES FOR SIMPLE BACKTRACK =  904
GUESSES FOR MRV =  0
GUESSES FOR AC3 =  0
GUESSES FOR WATERFALL =  0
puzzle/puz-025.txt
GUESSES FOR SIMPLE BACKTRACK =  69296
GUESSES FOR MRV =  0
GUESSES FOR AC3 =  0
GUESSES FOR WATERFALL =  0
puzzle/puz-026.txt
GUESSES FOR SIMPLE BACKTRACK =  7944
GUESSES FOR MRV =  18
GUESSES FOR AC3 =  18
GUESSES FOR WATERFALL =  0
puzzle/puz-048.txt
GUESSES FOR SIMPLE BACKTRACK =  93920
GUESSES FOR MRV =  20
GUESSES FOR AC3 =  20
GUESSES FOR WATERFALL =  2
puzzle/puz-051.txt
GUESSES FOR SIMPLE BACKTRACK =  7288
GUESSES FOR MRV =  13
GUESSES FOR AC3 =  13
GUESSES FOR WATERFALL =

In [7]:
countlister("puzzle/puz-002.txt")


GUESSES FOR SIMPLE BACKTRACK =  608
GUESSES FOR MRV =  0
GUESSES FOR AC3 =  0
GUESSES FOR WATERFALL =  0


In [89]:
def diffSolver(filename):
    puzzle = puzzle = SudokuPuzzle(possibleValues, filename)
    puzzle.updateBoardAndDomainValues()
    reached = puzzle.reachedGoal()
    # puzzle.print
    if(reached):
        return filename + "Light and Easy"
    i = 0
    while True:
        unassignedCount = puzzle.unAssignedValuesCnt()
        puzzle.nakedpairs();
        puzzle.updateBoardAndDomainValues()
        if(unassignedCount == puzzle.unAssignedValuesCnt()):
            break
    
    unassignedCount = puzzle.unAssignedValuesCnt()
    #puzzle.printAvailableValues();
    reached = puzzle.reachedGoal()
    if(reached):
        return filename + "Moderate"
    #puzzle.boxRemoveColumnAndRowValues();
    unassignedCount = puzzle.unAssignedValuesCnt()
    
    #puzzle.printBoard()
    count=0
    searchcount=0
    puzzle = SudokuPuzzle(possibleValues, filename)
    #puzzle.printBoard()
    bfFactor = puzzle.unAssignedValuesBranchFactor()
    print bfFactor*unassignedCount
    if(bfFactor*unassignedCount <= 125):
        return "HARD"
    puzzle = SolveUsingAC3(puzzle)
    #print unassignedCount
    #print searchcount - count
    #puzzle.printBoard()
    return "DONT EVEN TRY THEM Difficult ......"



filename=""
numbers=["puzzle/puz-001.txt","puzzle/puz-002.txt","puzzle/puz-010.txt","puzzle/puz-015.txt","puzzle/puz-025.txt","puzzle/puz-026.txt","puzzle/puz-048.txt","puzzle/puz-051.txt","puzzle/puz-062.txt","puzzle/puz-076.txt","puzzle/puz-081.txt","puzzle/puz-082.txt","puzzle/puz-090.txt","puzzle/puz-095.txt","puzzle/puz-099.txt","puzzle/puz-100.txt"]
#numbers=["puzzle/puz-081.txt"]
for i in numbers:
    print i
    global filename
    #filename=i
    print diffSolver(i)

puzzle/puz-001.txt
puzzle/puz-001.txtLight and Easy
puzzle/puz-002.txt
puzzle/puz-002.txtLight and Easy
puzzle/puz-010.txt
puzzle/puz-010.txtLight and Easy
puzzle/puz-015.txt
puzzle/puz-015.txtLight and Easy
puzzle/puz-025.txt
puzzle/puz-025.txtLight and Easy
puzzle/puz-026.txt
puzzle/puz-026.txtModerate
puzzle/puz-048.txt
puzzle/puz-048.txtModerate
puzzle/puz-051.txt
120.0
HARD
puzzle/puz-062.txt
122.777777778
HARD
puzzle/puz-076.txt
146.181818182
DONT EVEN TRY THEM Difficult ......
puzzle/puz-081.txt
205.49122807
DONT EVEN TRY THEM Difficult ......
puzzle/puz-082.txt
198.275862069
DONT EVEN TRY THEM Difficult ......
puzzle/puz-090.txt
126.879310345
DONT EVEN TRY THEM Difficult ......
puzzle/puz-095.txt
126.456140351
DONT EVEN TRY THEM Difficult ......
puzzle/puz-099.txt
163.090909091
DONT EVEN TRY THEM Difficult ......
puzzle/puz-100.txt
187.2
DONT EVEN TRY THEM Difficult ......
